# Attempt combining professor split and chi squared

In [1]:
import pandas as pd
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score

from sklearn.feature_selection import chi2, SelectPercentile

In [2]:
reviews = pd.read_csv("../Data/scraped_comments_with_professor.csv").sample(n=20000, random_state=0)

### Barebones preprocessing

In [3]:
reviews.drop_duplicates(subset="comment_id", keep="first", inplace=True)

reviews.dropna(subset=["comment"], inplace=True)
reviews = reviews[reviews["comment"] != "No Comments"]

reviews["comment"] = reviews["comment"].apply(lambda x: x if len(x.split()) > 5 else None)
reviews.dropna(subset=["comment"], inplace=True)

reviews.reset_index(drop=True, inplace=True)

In [4]:
comments_proper = []

for i in range(reviews.shape[0]):
    review = reviews["comment"][i]
    review = re.sub('&([a-zA-z]+|#\d+);', "", review)           # remove HTML codes
    review = re.sub('&#63;?', '', review)                       # HTML code for question mark evades erasure on occasion, handle here
    review = re.sub(r'\s*https?://\S+(\s+|$)', ' ', review)                                     # remove links
    review = re.sub("^(\+\d{1,2}\s)?\(?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}$", ' ', review)         # remove phone numbers
    review = re.sub("[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+", " ", review)              # remove email addresses

    review = re.sub(r'(.)\1\1+', '\g<1>', review)               # replace any three characters in a row with one

    review = re.sub('[^a-zA-Z]+', ' ', review)                  # remove non-alphabetic characters

    review = re.sub('\s+', ' ', review)
    review = review.lower()                                     # lowercase review for uniformity

    comments_proper.append(review)

In [5]:
reviews = reviews.loc[:, ["professor_id", "firstName", "lastName", "comment", "clarityRating"]]
reviews["cleanedComment"] = pd.Series(comments_proper)
reviews["sentiment"] = reviews["clarityRating"].apply(lambda x: 1 if x > 2.5 else 0)

reviews.head()

,professor_id,firstName,lastName,comment,clarityRating,cleanedComment,sentiment
0,VGVhY2hlci0xNDYwNTM2,Heather,Kruse,She is so incredible and is always willing to ...,5,she is so incredible and is always willing to ...,1
1,VGVhY2hlci0xMjkzMDkw,Joseph,Lavalle,Awesome Teacher. Highly Recommended. Funny guy...,5,awesome teacher highly recommended funny guy a...,1
2,VGVhY2hlci0xMjEwNzE2,John,Park,Honestly one of the best compsci professors at...,5,honestly one of the best compsci professors at...,1
3,VGVhY2hlci0xMzQyNDg4,Thomas,Buford,He is an excellent teacher. He puts all of th...,4,he is an excellent teacher he puts all of the ...,1
4,VGVhY2hlci0xMDcxMDM2,Shawn,Kenny,Fair prof. Very thorough in his coverage of th...,4,fair prof very thorough in his coverage of the...,1


### Perform professor split

In [6]:
prof_counts = reviews["professor_id"].value_counts()
prof_train, prof_test, cnt_train, cnt_test = train_test_split(prof_counts.index, prof_counts.values, test_size=0.2, random_state=1)

In [7]:
comm_train = reviews[reviews["professor_id"].isin(prof_train)]["cleanedComment"]
comm_test = reviews[reviews["professor_id"].isin(prof_test)]["cleanedComment"]

In [8]:
sent_train = reviews[reviews["professor_id"].isin(prof_train)]["sentiment"]
sent_test = reviews[reviews["professor_id"].isin(prof_test)]["sentiment"]

In [9]:
def evalPerformance(y_pred, y_test, mode="weighted"):
    acc_score = accuracy_score(y_test, y_pred)
    print("Accuracy Score: " + str(acc_score * 100))
    
    f1 = f1_score(y_test, y_pred, average=mode)
    print("F1 Score: {0}".format(f1 * 100))

### Prepare vectorizer and chi squared feature selector

In [10]:
cv = CountVectorizer(ngram_range=(1,2))
X = cv.fit_transform(reviews["cleanedComment"]).toarray()

In [11]:
percbest = SelectPercentile(score_func=chi2, percentile=25)      # select 25% most important features
percbest.fit_transform(X, reviews["sentiment"])

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

### Create naive bayes model and evaluate performance

In [12]:
X_train = percbest.transform(cv.transform(comm_train))      # get training vector from top 25% of features
model = MultinomialNB().fit(X_train, sent_train)            # train model

In [13]:
X_test = percbest.transform(cv.transform(comm_test))
sent_pred = model.predict(X_test)

In [14]:
evalPerformance(sent_pred, sent_test)

Accuracy Score: 93.4404636459431
F1 Score: 93.2296792992565


##### Pickle model so it can be used in other files

In [19]:
import pickle
from joblib import dump, load

"""
with open("NB_model.pickle", "w+") as f:
    pickle.dump(model, f, protocol=5)
"""

# turns out joblib is better for persisting large numpy arrays,
# which are quite common in ML models

dump(model, "NB-model.joblib")
dump(cv, "NB-vectorizer.joblib")
dump(percbest, "NB-chi2.joblib")

['NB-chi2.joblib']